In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

import spacy
import dask.dataframe as dd

import os
os.chdir('/Users/benjaminforleo/Box/spring_project/')

In [ ]:
df = pd.read_csv('ira_tweets_csv_hashed.csv', low_memory = False)

In [ ]:
df.columns

In [ ]:
# FILTER DOWN TO ENGLISH TWEETS AND A SUBSET OF COLUMNS
en_tweets = df[df.tweet_language == "en"]

col_list = ['tweetid', 'userid','tweet_text', 'tweet_time']

en_tweets = en_tweets.loc[:,col_list]

In [ ]:
# ADD A HASHTAG PIPE TO THE SPACY PIPELINE 
def hashtag_pipe(doc):
    merged_hashtag = False
    while True:
        for token_index, token in enumerate(doc):
            if token.text == '#':
                if token.head is not None:
                    start_index = token.idx
                    end_index = start_index + len(token.head.text) + 1
                    if doc.merge(start_index, end_index) is not None:
                        merged_hashtag = True
                        break
        if not merged_hashtag:
            break
        merged_hashtag = False
    return doc

nlp = spacy.load('en')
nlp.add_pipe(hashtag_pipe)

doc = nlp("twitter #hashtag")
assert len(doc) == 2
assert doc[0].text == 'twitter'
assert doc[1].text == '#hashtag'

In [ ]:
# add a retweet pipe

In [ ]:
# PROCESS THE TEXT IN PARALLEL
ddata = dd.from_pandas(en_tweets[['tweet_text']][:100], npartitions=4)

res = ddata.map_partitions(lambda df: df.apply((lambda row: nlp(*row)),
                                               axis=1)).compute(scheduler = 'threads')  